<a href="https://colab.research.google.com/github/Nadine-001/KlasifikasiGambar/blob/master/Klasifikasi%20Gambar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Submission Kelas "Belajar Machine Learning untuk Pemula"
# Nama : Vanessa Sisilia Dewi
# Domisili : Kota Semarang, Jawa Tengah
# Email : na.nadine.001@gmail.com

In [ ]:
# mengimpor library yang diperlukan
import zipfile, shutil, os
import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, Activation, ReLU
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# mengimpor dataset
!wget --no-check-certificate \
https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip \
-O /tmp/rock_paper_scissors.zip

--2021-05-11 11:49:46--  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip
Resolving dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)... 52.239.197.36
Connecting to dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)|52.239.197.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322873683 (308M) [application/zip]
Saving to: ‘/tmp/rock_paper_scissors.zip’

/tmp/rock_paper_sci 100%[===================>] 307.92M  15.8MB/s    in 19s     

2021-05-11 11:50:05 (16.2 MB/s) - ‘/tmp/rock_paper_scissors.zip’ saved [322873683/322873683]



In [ ]:
# mengekstrak file
local_zip = '/tmp/rock_paper_scissors.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [ ]:
# mengecek isi directori
os.listdir('/tmp/rockpaperscissors')

['rps-cv-images', 'README_rpc-cv-images.txt', 'rock', 'scissors', 'paper']

In [ ]:
base = '/tmp/rockpaperscissors'

# menghapus file dan direktori di bawah beserta isinya karena
# tidak terlalu penting
os.remove(os.path.join(base, 'README_rpc-cv-images.txt'))
shutil.rmtree(os.path.join(base, 'rps-cv-images'))

# membagi dan membuat direktori
train = os.path.join(base, 'train')
validation = os.path.join(base, 'val')

In [ ]:
# membuat direktori tangan yang membentuk batu pada direktori data training dan
# data validasi
train_roc = os.path.join(train, 'rock')
validation_roc = os.path.join(validation, 'rock')
 
# membuat direktori tangan yang membentuk kertas pada direktori data training dan
# data validasi
train_pap = os.path.join(train, 'paper')
validation_pap = os.path.join(validation, 'paper')

# membuat direktori tangan yang membentuk gunting pada direktori data training dan
# data validasi
train_sci = os.path.join(train, 'scissor')
validation_sci = os.path.join(validation, 'scissor')

In [ ]:
train_datagen = ImageDataGenerator(
                    validation_split=0.40,
                    rescale=1./255,
                    rotation_range=45,
                    brightness_range=[0.1,1.0],
                    shear_range=0.2,
                    zoom_range=0.5,
                    horizontal_flip=True,
                    vertical_flip=True,
                    fill_mode='nearest')

test_datagen = ImageDataGenerator(
                    validation_split=0.40,
                    rescale=1./255,
                    rotation_range=45,
                    brightness_range=[0.1,1.0],
                    shear_range=0.2,
                    zoom_range=0.5,
                    horizontal_flip=True,
                    vertical_flip=True,
                    fill_mode='nearest')

In [ ]:
# membagi train generator dan validation generator

train_gen = train_datagen.flow_from_directory(
        base,
        target_size=(150,150),
        batch_size=32,
        color_mode='rgb',
        class_mode='categorical',
        subset='training')
 
validation_gen = test_datagen.flow_from_directory(
        base,
        target_size=(150,150),
        batch_size=32,
        color_mode='rgb',
        class_mode='categorical',
        subset='validation')

Found 1314 images belonging to 3 classes.
Found 874 images belonging to 3 classes.


In [ ]:
model = tf.keras.models.Sequential()

# convolutional layer pertama
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))

# convolutional layer kedua
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))

# convolutional layer ketiga
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))

# convolutional layer keempat
model.add(Conv2D(256, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten(input_shape=(150,150,3)))
model.add(Dense(256, activation=ReLU(negative_slope=0.5)))
model.add(Dense(256, activation=ReLU(negative_slope=0.5)))
model.add(Dense(512, activation=ReLU(negative_slope=0.5)))
model.add(Dropout(0.3)) 
model.add(Dense(3, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 146, 146, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 69, 69, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 34, 34, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 128)       7

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [ ]:
model.fit(
     train_gen,
     steps_per_epoch=train_gen.n//train_gen.batch_size,
     epochs=20,
     validation_data=validation_gen,
     validation_steps=validation_gen.n//validation_gen.batch_size,
     verbose=2)

Epoch 1/20
41/41 - 51s - loss: 1.1009 - accuracy: 0.3354 - val_loss: 1.0968 - val_accuracy: 0.3715
Epoch 2/20
41/41 - 19s - loss: 1.0991 - accuracy: 0.3557 - val_loss: 1.0977 - val_accuracy: 0.3449
Epoch 3/20
41/41 - 19s - loss: 1.0830 - accuracy: 0.3846 - val_loss: 1.0709 - val_accuracy: 0.4896
Epoch 4/20
41/41 - 19s - loss: 0.9982 - accuracy: 0.4938 - val_loss: 0.9786 - val_accuracy: 0.5590
Epoch 5/20
41/41 - 19s - loss: 0.8428 - accuracy: 0.6381 - val_loss: 0.6174 - val_accuracy: 0.7350
Epoch 6/20
41/41 - 19s - loss: 0.5945 - accuracy: 0.7473 - val_loss: 0.6220 - val_accuracy: 0.7546
Epoch 7/20
41/41 - 19s - loss: 0.5947 - accuracy: 0.7559 - val_loss: 0.5198 - val_accuracy: 0.8056
Epoch 8/20
41/41 - 19s - loss: 0.5454 - accuracy: 0.7941 - val_loss: 0.5397 - val_accuracy: 0.7928
Epoch 9/20
41/41 - 19s - loss: 0.5031 - accuracy: 0.7956 - val_loss: 0.5008 - val_accuracy: 0.7963
Epoch 10/20
41/41 - 19s - loss: 0.4967 - accuracy: 0.7949 - val_loss: 0.4513 - val_accuracy: 0.8206
Epoch 11/

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
 
uploaded = files.upload()
 
for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(150,150))
  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
 
  images = np.vstack([x])

  resultProb = model.predict(images) 
  resultLabel = resultProb.argmax(axis=-1)

  # printing images based on class
  if resultLabel == 0:
    plt.title("Result: Rock")
  elif resultLabel == 1:
    plt.title("Result: Paper")
  elif resultLabel == 2:
    plt.title("Result: Scissor")